In [1]:
import numpy as np

from networks import FiniteFieldPiNetNetworkLinear, FiniteFieldPiNetNetworkLeNet, FiniteFieldPiNetNetworkLeNetCIFAR10
from criterions import FiniteFieldMSELoss
from datasets import load_all_data_mnist, load_all_data_cifar10
from utils import create_batch_data, to_real_domain

In [2]:
BATCH_SIZE = 128
EPOCH = 1
LR = 8
PRINT = 10
MODE = 3
PRIME = 2**26 - 5
QUANTIZATION_WEIGHT = 8
QUANTIZATION_INPUT = 8
QUANTIZATION_BATCH_SIZE = 7

In [3]:
if MODE == 1:
    model = FiniteFieldPiNetNetworkLinear(QUANTIZATION_WEIGHT, PRIME, QUANTIZATION_INPUT)
    flatten = True
elif MODE == 2:
    model = FiniteFieldPiNetNetworkLeNet(QUANTIZATION_WEIGHT, PRIME, QUANTIZATION_INPUT)
    flatten = False
elif MODE == 3:
    model = FiniteFieldPiNetNetworkLeNetCIFAR10(QUANTIZATION_WEIGHT, PRIME, QUANTIZATION_INPUT)
    flatten = False
else:
    model = None
    flatten = True
criterion = FiniteFieldMSELoss(PRIME, QUANTIZATION_WEIGHT, QUANTIZATION_BATCH_SIZE)

In [4]:
# data fetching
load_path = '../../data'
train_data, train_label, test_data, test_label = load_all_data_cifar10(load_path, QUANTIZATION_INPUT, QUANTIZATION_WEIGHT, PRIME, flatten=flatten)
train_data, train_label, test_data, test_label = create_batch_data(train_data, train_label, test_data, test_label, BATCH_SIZE)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
for epoch in range(EPOCH):
    tot_loss = 0
    for train_idx, (train_data_batch, train_label_batch) in enumerate(zip(train_data, train_label)):
        # train
        preds = model.forward(train_data_batch)

        loss = criterion.forward(preds, train_label_batch)
        tot_loss += loss
        propagated_error = criterion.error_derivative()

        model.backprop(propagated_error)
        model.optimize(LR)
        print('idx: {}, loss: {}'.format(train_idx + 1, loss))
        if (train_idx + 1) % PRINT == 0:
            print('epoch: {}, idx: {}, loss: {}'.format(epoch + 1, train_idx + 1, tot_loss / PRINT))

        # if (train_idx + 1) % PRINT == 0:
        #     tot_acc = 0
        #     tot_sample = 0
        #     for train_acc_idx, (test_data_batch, test_label_batch) in enumerate(zip(test_data, test_label)):
        #         # train accuracy
        #         preds = model.forward(test_data_batch)
        #         real_preds = to_real_domain(preds, QUANTIZATION_WEIGHT, PRIME)
        #         pred_args = np.argmax(real_preds, axis=1)
        #
        #         tot_acc += np.count_nonzero(pred_args == test_label_batch)
        #         tot_sample += test_data_batch.shape[0]
        #
        #     accuracy = tot_acc / tot_sample
        #     if train_idx != 0:
        #         tot_loss = tot_loss / PRINT
        #     print('epoch: {}, idx: {}, accuracy: {}, loss: {}'.format(epoch + 1, train_idx + 1, accuracy, tot_loss))
        #     tot_loss = 0